In [ ]:
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from Augmentor import Pipeline

epochs = 10
batch_size = 256
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
def augment_images(data_path: str, augment_count: int) -> None:

    sample = Augmentor.Pipeline(data_path)
    Pipeline.set_seed(1230)
    sample.resize(probability=1.0, width=160, height=120, resample_filter="BICUBIC")
    sample.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
    sample.zoom(probability=0.5, min_factor=1.0, max_factor=1.2)
    sample.flip_left_right(probability=0.5)
    sample.gaussian_distortion(
        probability=1,
        grid_width=4,
        grid_height=4,
        magnitude=2,
        corner="bell",
        method="in",
        mex=0.5,
        mey=0.5,
        sdx=0.05,
        sdy=0.05,
    )
    sample.random_brightness(probability=1, min_factor=0.5, max_factor=1.5)
    sample.random_color(probability=1, min_factor=0.5, max_factor=2)

    sample.status()
    sample.sample(augment_count, multi_threaded=True)
    return

In [ ]:
# EXAMPLE AUGMENTATION FOR TRAINING DATASET
augment_images('/home/z6/Furkan-data/seatbelt/REFINED_RAW_DATASET/train', 300_000)

In [ ]:
# 300_000 AUGMENTED DATA
TRAIN_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original/train'
VAL_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original/val'
TEST_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original//test'

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

batch_size = 256

train_datagen = ImageDataGenerator(rescale=1 / 255.0)


trainGen = train_datagen.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size)

val_datagen = ImageDataGenerator(rescale=1 / 255.0)

valGen = val_datagen.flow_from_directory(
    VAL_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size)

testAug = ImageDataGenerator(rescale=1 / 255.0)

testGen = testAug.flow_from_directory(
    TEST_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size)

In [ ]:
#CUSTOM CALLBACK TO PRINT LOSS AND ACCURACY DURING TRAINING 
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.4f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print(" For batch {}, loss is {:7.4f}.".format(batch, logs["loss"]))


In [ ]:
# GOOGLENET MODEL COMPILE AND TRAIN


model = tf.keras.applications.InceptionV3(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

In [ ]:
# CLASSIFICATION REPORT AND THE CONFUSION MATRIX FOR THE GOOGLENET
testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))

In [ ]:
# DENSENET121 MODEL COMPILE AND TRAIN

model = tf.keras.applications.DenseNet121(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

In [ ]:
# CLASSIFICATION REPORT AND THE CONFUSION MATRIX FOR DENSENET121

testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))

In [ ]:
# RESNET50 MODEL COMPILE AND TRAIN

model = tf.keras.applications.ResNet50(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

In [ ]:
# CLASSIFICATION REPORT AND THE CONFUSION MATRIX FOR RESNET50

testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))